In [39]:
# Reference : 
# https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/

import sys
import os

# os.environ['THEANO_FLAGS'] = "device=cuda*"
os.environ['KERAS_BACKEND']='tensorflow'
os.environ['MKL_THREADING_LAYER']='GNU'

import numpy as np
import pandas as pd
from collections import defaultdict
import re

In [40]:
import urllib
import urllib.parse
import urllib.request

data = {'q': 'Sheraton and Pan Am said they are assured under the Soviet joint-venture law that they can repatriate profits from their hotel venture.'}
params = urllib.parse.urlencode(data)
url = "http://138.197.118.157:8000/segbot/api/?%s" % params
response = urllib.request.urlopen(url).read()
print(response)

HTTPError: HTTP Error 404: Not Found

In [3]:
import numpy as np

In [8]:
response = np.asarray(response)

In [9]:
response.shape

()

In [10]:
response

array(b'"[\\"Sheraton and Pan Am said\\", \\"they are assured under the Soviet joint-venture law\\", \\"that they can repatriate profits from their hotel venture .\\"]"',
      dtype='|S154')

In [4]:
import pickle

def save_pickle(path, X):
    with open(path, 'wb') as f:
        pickle.dump(X, f)

def open_pickle(path):
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train = open_pickle("../../data/imdb/imdb_original_preprocessed_xtrain.pickle")
X_test = open_pickle("../../data/imdb/imdb_original_preprocessed_xtest.pickle")
y_train = open_pickle("../../data/imdb/imdb_original_preprocessed_ytrain.pickle")
y_test = open_pickle("../../data/imdb/imdb_original_preprocessed_ytest.pickle")

In [5]:
# data = {'q': 'Sheraton and Pan Am said they are assured under the Soviet joint-venture law that they can repatriate profits from their hotel venture.'}
data = {'q' : X_train[0]}
params = urllib.parse.urlencode(data)
url = "http://138.197.118.157:8000/segbot/api/?%s" % params
response = urllib.request.urlopen(url).read()

In [13]:
X_train[0]

'silent night, deadly night 5 is the very last of the series, and like part 4, it is unrelated to the first three except by title and the fact that it is a christmas-themed horror flick.except to the oblivious, there is some obvious thing going on here...mickey rooney plays a toymaker named joe petto and his creepy son\'s name is pino. ring a bell, anyone? now, a little boy named derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. even though it said "do not open till christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. inside is a little red ball that sprouts santa arm and a head, and proceed to kill dad. oop, maybe he should have left well-enough alone. of course derek is then traumatized by the incident since he watched it from the stair, but he does not grow up to be some killer santa, he just stops talking.there is a mysterious stranger lurking around, w

In [14]:
response

b'"[\\"silent night , deadly night 5 is the very last of the series ,\\", \\"and like part 4 , it is unrelated to the first three except by title\\", \\"and the fact\\", \\"that it is a christmas-themed horror flick.except to the oblivious ,\\", \\"there is some obvious thing\\", \\"going on here ...\\", \\"mickey rooney plays a toymaker\\", \\"named joe petto\\", \\"and his creepy son \'s name is pino .\\", \\"ring a bell , anyone ?\\", \\"now , a little boy named derek heard a knock at the door one evening ,\\", \\"and opened it to find a present on the doorstep for him .\\", \\"even though it said\\", \\"`` do not open till christmas \'\' ,\\", \\"he begins to open it anyway\\", \\"but is stopped by his dad ,\\", \\"who scolds him and sends him to bed ,\\", \\"and opens the gift himself .\\", \\"inside is a little red ball\\", \\"that sprouts santa arm and a head ,\\", \\"and proceed to kill dad .\\", \\"oop , maybe he should have left well-enough alone .\\", \\"of course derek is t

In [88]:
import json

In [89]:
output = json.loads(response)

In [90]:
output

'["Sheraton and Pan Am said", "they are assured under the Soviet joint-venture law", "that they can repatriate profits from their hotel venture ."]'

In [91]:
output[0]

'['

In [94]:
output[1]

'"'

In [92]:
from textblob import TextBlob

In [93]:
text = TextBlob(output)

In [95]:
output = output[1:len(output)-1]

In [96]:
output

'"Sheraton and Pan Am said", "they are assured under the Soviet joint-venture law", "that they can repatriate profits from their hotel venture ."'

In [71]:
output = output[:len(output)-1]

In [72]:
output

'"Sheraton and Pan Am said", "they are assured under the Soviet joint-venture law", "that they can repatriate profits from their hotel venture ."'

In [80]:
test = output.split(", ")

In [81]:
test

['"Sheraton and Pan Am said"',
 '"they are assured under the Soviet joint-venture law"',
 '"that they can repatriate profits from their hotel venture ."']

In [82]:
len(test)

3

In [83]:
test[0]

'"Sheraton and Pan Am said"'

In [84]:
test[1]

'"they are assured under the Soviet joint-venture law"'

In [109]:
test[0].replace("\"", "")

'Sheraton and Pan Am said'

In [16]:
def load_imdb(path, shuffle=True, random_state=42):
    import glob 
    print("Loading the imdb data")
    
    train_neg_files = glob.glob(path+"/train/neg/*.txt")
    train_pos_files = glob.glob(path+"/train/pos/*.txt")
    
#     X_train_corpus = []
    X_train_pos = []
    X_train_neg = []
#     y_train = []
    
    for tnf in train_neg_files:
        f = open(tnf, 'r', encoding="utf8")
        line = f.read()
        X_train_neg.append(line)
#         y_train.append(0)
        f.close()
    
    for tpf in train_pos_files:
        f = open(tpf, 'r', encoding="utf8")
        line = f.read()
        X_train_pos.append(line)
#         y_train.append(1)
        f.close()
    
    print("Train Data loaded.")
    
    test_neg_files = glob.glob(path+"/test/neg/*.txt")
    test_pos_files = glob.glob(path+"/test/pos/*.txt")
    
#     X_test_corpus = []
#     y_test = []
    X_test_pos = []
    X_test_neg = []
    
    for tnf in test_neg_files:
        f = open(tnf, 'r', encoding="utf8")
        X_test_neg.append(f.read())
#         y_test.append(0)
        f.close()
    
    for tpf in test_pos_files:
        f = open(tpf, 'r', encoding="utf8")
        X_test_pos.append(f.read())
#         y_test.append(1)
        f.close()
    
    print("Test Data loaded.")
    
#     y_train = np.array(y_train)
#     y_test = np.array(y_test)
    
    return X_train_pos, X_train_neg, X_test_pos, X_test_neg

In [17]:
path = r"C:\Users\Anneke\Documents\GitHub\data\aclImdb"
X_train_pos, X_train_neg, X_test_pos, X_test_neg = load_imdb(path)

Loading the imdb data
Train Data loaded.
Test Data loaded.


In [18]:
len(X_train_pos)

12500

In [7]:
def clean_edu(edus):
    import json
    output = json.loads(edus)
    
    # eliminate bracket
    output = output[1:len(output)-1]
    
    # split edu with delimiter
    edu = output.split(", ")
    del output
    
    for i in range(len(edu)):
        edu[i] = edu[i].replace("\"", "")
#         print(item)
    
    return edu

In [36]:
edu_path = "./train/"
import time
start = 15017
# 49 error
# 69 error
# 71 error
# 73
# 77 
# 87
# 94 95
error_indices = []
for i,corpus in enumerate(X_train[start:20000]):
    i = i + start
    data = {'q' : corpus}
    params = urllib.parse.urlencode(data)
    url = "http://138.197.118.157:8000/segbot/api/?%s" % params
#     response = urllib.request.urlopen(url).read()
    try:
        response = urllib.request.urlopen(url).read()
    except urllib.error.HTTPError:
        error_indices.append(i)
        continue
        
    edu = clean_edu(response)
    
    f = open(edu_path + str(i) + "_" + str(len(edu)) + ".txt", "w")
    
    for j in range(len(edu)):
        if i == (len(edu)-1):
            f.write(edu[j])
            break
        f.write(edu[j]+"\n")
    f.close()
    
    print(i)
    if i%100 == 0:
        time.sleep(100)
    elif i%10 == 0:
        time.sleep(60)

In [37]:
len(error_indices)

f = open("./error_indices.txt", "a")
for i in error_indices:
    f.write(str(i) + "\n")
f.close()

In [38]:
error_indices

[15017,
 15018,
 15019,
 15020,
 15021,
 15022,
 15023,
 15024,
 15025,
 15026,
 15027,
 15028,
 15029,
 15030,
 15031,
 15032,
 15033,
 15034,
 15035,
 15036,
 15037,
 15038,
 15039,
 15040,
 15041,
 15042,
 15043,
 15044,
 15045,
 15046,
 15047,
 15048,
 15049,
 15050,
 15051,
 15052,
 15053,
 15054,
 15055,
 15056,
 15057,
 15058,
 15059,
 15060,
 15061,
 15062,
 15063,
 15064,
 15065,
 15066,
 15067,
 15068,
 15069,
 15070,
 15071,
 15072,
 15073,
 15074,
 15075,
 15076,
 15077,
 15078,
 15079,
 15080,
 15081,
 15082,
 15083,
 15084,
 15085,
 15086,
 15087,
 15088,
 15089,
 15090,
 15091,
 15092,
 15093,
 15094,
 15095,
 15096,
 15097,
 15098,
 15099,
 15100,
 15101,
 15102,
 15103,
 15104,
 15105,
 15106,
 15107,
 15108,
 15109,
 15110,
 15111,
 15112,
 15113,
 15114,
 15115,
 15116,
 15117,
 15118,
 15119,
 15120,
 15121,
 15122,
 15123,
 15124,
 15125,
 15126,
 15127,
 15128,
 15129,
 15130,
 15131,
 15132,
 15133,
 15134,
 15135,
 15136,
 15137,
 15138,
 15139,
 15140,
 15141,


In [24]:
X_train[69].replace("\'","")

'sista in da hood. looking for revenge and bling bling. except da hood is a wild west town in the late 1800. i do not remember any western like this when i was growing up. what would randolph scott say? if he saw lil kim, he might say, "alright! i have to admit that i tuned into this just to see her. bare midriff and low cut blouse are not the staple of the usual cowboy flick, but these are the cowgirls, and they are fine.now, do not go looking for any major story here, and the usual stuff of ghetto crime drama are here in a different setting. and, whens the last time you heard john wayne call someone, "dawg"? and, i do not remember the earp other hugging and kissing before they marched to the ok corral.i watch this on bet, so i missed the action that got it an r rating, but i doubt if i will buy the dvd to see it unless i can be assured it was lil kim in that action.'

In [14]:
X_train[48]

"i pity person calling kamal hassan 'ulaganaayakan' maybe for them ulagam is tollywood ! comeon guys..this movie is a thriller without thrill..come out of your ulagam and just watch some high class thriller like the usual suspects or even the silence of the lambs.technically good but style over substance kamal does not look like a police officer, there is no thrill whatsoever dragging and boring till end you might be saving 3 valuable hrs of your life if u skip watching this movie.kamal at his best is the best in tollywood"

In [26]:
X_train[71]

'it has been about 50 year since a movie has been made about romance and mysticism. the only two movie i can think of is "enchanted april" (1992) and "the enchanted cottage" (1945). both movie used wonderful actor not star. in both movie, all the actor gave their best romantic performance."enchanted april" is about four english woman after wwi who are unhappy with their life and find happiness in italy while on vacation. it is amazing "enchanted april" was made in 1992. it stands out as an enjoyable classic.'

In [29]:
X_train[73]

'spoiler... if such a thing is possible... . . . . . . . as a rabid robin william fan, i felt it necessary to buy this film as first on-screen appearance. wow... i could not imagine a more mind-numbing movie. essentially, the movie takes one bad joke after another that your uncle artie would tell you after dinner and dramatizes them. robin william plays a lawyer in a 30 sec skit.i am all for bawdy humor, but this humor was not pleasantly vulgar, or ribald... it was just mind-numbing. there are no redeeming quality to this film, other than robin william fanatic, like me, who simply have to own every piece of film.'

In [44]:
X_train[5]

"when i was a kid we always used to be babysat, and we always used to rent a film or see a film at the cinema. this is one of the film we watched. this is one of the stupidest film i have ever seen, i think it might even be a walt disney picture film! a martian is dropped on earth, turns into a human, befriends a human, and is trying everything he can to get back home. but he is distracted by the wonder of the earth. the only good comment i can give is the choice of actor, back to the future's christopher lloyd as the martian, uncle martin, dumb and dumber's jeff daniel as tim o'hara, elizabeth hurley as brace channing and daryl hannah as lizzie. but apart from that it is complete crap. poor!"

In [47]:
y_train[5]

0

In [45]:
X_train[6]

"1st watched 12/7/2002 - 3/10(dir-steve purcell): typical mary kate and ashley fare with a few more kiss. it looks to me like the girl are getting pretty tired of this stuff and it will be interesting what happens to them if they ever decide to split up and go there own way. in this episode of their adventure they are intern in rome for a `fashion' designer who puts them right into the mailroom to learn what working hard is all about(i guess..). besides the typical flirtation with boy there is nothing much else except the rome scenario until about ¾ way into the movie when it is finally revealed why they are getting fired, then re-hired, then fired again, then re-hired again. this is definetly made by person who do not understand the corporate world and it shows in their interpretation of it. maybe the real world will be their next adventure(if there is one.). even my kid did not seem to care for this boring `adventure' in the make-believe. let's see they probably only have a couple of

In [48]:
y_train[6]

0

In [46]:
X_train[7]

'"a cry in the dark" is a masterful piece of cinema, haunting, and incredibly though provoking. the true story of lindy chamberland, who, in 1980, witnessed a horrific sight, seeing her 3-month-old baby being utally taken from their family\'s tent, while camping on the austrailian outback. azaria (the baby) was never seen again, and the result of her horrendous disappearance caused a true life frenzy all around the world. meryl streep does immaculate justice to the role of lindy, as she always does. but the one thing that helps "a cry in the dark" never fall flat is the illiant direction. a truly inspired and accurate outlook on this baffeling case, tear are ought to the eye. the concept is nothing less then terrifying, and afterward you are left haunted, but also inspired.'

In [50]:
y_train[7]

1

In [51]:
y_train[9]

0